In [256]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [257]:
# read from melodyData.txt
with open('melodyData.txt', 'r') as f:
    parts = eval(f.read())
# a =[(0, '0.25'), (0, '1.0'), (0, '3.0'), (70, '.33'), ...]


# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


In [258]:
sequence_length = 10

X = []
Y = []
# given data_encoded, generate training data by looping 
for i in range(len(data_encoded)):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + 10])
        Y.append(data_encoded[i][j + 10])

# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# TODO: separate different parts into train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# increase learning rate
tf.keras.optimizers.Adam(learning_rate=0.1)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(sequence_length,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='softplus'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=200)

Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1529 - loss: 4.1385    
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2744 - loss: 3.3971 
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3216 - loss: 2.9667 
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3312 - loss: 2.7966 
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3542 - loss: 2.6005 
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4058 - loss: 2.3630 
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4092 - loss: 2.2639 
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4160 - loss: 2.1753 
Epoch 9/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4171 - loss: 2.1401 
Epoch 10/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4416 - loss: 1.9309 
Epoch 11/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5052 - loss: 1.7690 
Epoch 12/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

In [259]:
# given a sequence, generate the next note (up to ten times)
seq = X_test[0]
total_seq = seq[:]
for i in range(6):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    seq = seq[1:] + [pred]

    total_seq.append(pred)

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}

for note in total_seq:
    pitch, dur = decodings[note]
    print (mapping[pitch % 12], dur)

# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
C 0.25
A# 0.50
C 0.25
A 0.25
A# 1.25
C 0.75
C 1.00
A# 0.50
C 0.25
A# 0.25
C 0.25
A 0.25
C 0.25
A 0.25
C 0.25
A 0.25


In [260]:
l = []
for i in range(10):
    l.append(i)
print (l)

a = [i*i for i in l]
print (a)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
